In [10]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Mac And Cheese Garlic Bread Bowl

1 italian bread
¾ stick unsalted butter
6 stick garlic cloves
3 tablespoons fresh parsley
¼ teaspoon McCormick® red pepper flakes
1 ½ teaspoons kosher salt
1 ½ tablespoons olive oil
½ cup grated parmesan cheese
1 ½ cups heavy cream
¾ cup shredded low-moisture mozzarella
¼ teaspoon freshly ground black pepper
2 ½ cups elbow macaroni


1. Preheat the oven to 400°F (200°C).
2. Cut off the top of the boule and reserve. Using a small, sharp knife, cut around the inside of the bread (being careful not to cut through the sides) and scoop out the center. Tear 1 loosely packed cup of the bread’s innards into 1-inch pieces, then discard or set aside the rest for another use.
3. In a small bowl, mix together 6 tablespoons softened butter, 3 cloves garlic, 1 tablespoon parsley, the red pepper flakes, if using, and ¼ teaspoon salt.
4. Arrange the bread bowl, top of the boule, and the torn bread on a baking sheet. Spread the compound butter over the inside of the to

In [11]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe2.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


--- MODEL TRANING ---

In [12]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, config=config)


In [14]:

file_name = "recipe2.csv"
ai.save("aitextgen.model")
train_tokenizer(file_name)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
ai.train(data, batch_size=2000, num_steps=64000, generate_every=1000, save_every=5000)
ai.generate(prompt="steak")

#Save the model
ai.save("aitextgen.model")

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.


Loss: 1.550 — Avg: 1.547 — GPU Mem: 14527 MB:  21%|██        | 13420/64000 [49:49<3:07:49,  4.49it/s]


100%|██████████| 9922/9922 [01:20<00:00, 122.82it/s]
pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loss: 1.550 — Avg: 1.543 — GPU Mem: 13932 MB:   1%|▏         | 840/64000 [03:04<3:51:34,  4.55it/s]